In [13]:
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
from supabase import create_client



dotenv_path = Path('.env')
load_dotenv(dotenv_path=dotenv_path)

url: str = os.getenv('SUPABASE_URL')
key: str = os.getenv('SUPABASE_KEY')


def init():
    supabase = create_client(url, key)
    return supabase

def read_data():      
      supabase = init()
      response = supabase.table('muaban').select("*").execute()
      df_muaban = pd.DataFrame(response.data)
      response = supabase.table('mogi').select("*").execute()
      df_mogi = pd.DataFrame(response.data)
      response= supabase.table('rongbay').select("*").execute()
      df_rongbay = pd.DataFrame(response.data)
      df_concatenated  = pd.concat([df_mogi, df_muaban, df_rongbay], ignore_index=True)
      df_concatenated['id'] = range(1, len(df_concatenated) + 1)
      return df_concatenated
      
df = read_data()

## Preprocessing


In [15]:
import json
from random import randint


with open('location.json', 'r', encoding='utf-8') as file:
    location = json.load(file)

def get_district_name_by_ward(location, ward_name):
    for district in location["district"]:
        if ward_name in district["wards"]:
            return district["name"]
    return None 

def get_ward_by_street(location, street_name):
    # Duyệt qua các district trong location
    for district in location["district"]:
        if street_name in district["streets"]:
            index = district["streets"].index(street_name)
            if index < len(district["wards"]):
                return district["wards"][index]
    return None  

def get_street_by_ward(location, ward):
    for district in location["district"]:
        if ward in district["wards"]:
            index = len(district['streets'])
            return district["streets"][randint(0, index-1)]




for i in range(len(df)):
    if df.loc[i, 'district'] == '':
            ward = df.loc[i, 'ward']
            df.loc[i, 'district'] = get_district_name_by_ward(location, ward)


    if df.loc[i,'ward'] == '':
        street = df.loc[i, 'street']
        df.loc[i,'street'] = get_ward_by_street(location, street)

    if df.loc[i,'street'] == '':
        ward= df.loc[i,'ward']
        df.loc[i,'street'] = get_street_by_ward(location, ward)
    if df.loc[i, 'direction'] == '':
        df.loc[i, 'direction'] = 0
    if df.loc[i,'price'] == 0.0:
        df = df.drop(i)



In [4]:
df.to_csv('data.csv', index=False)
df.head()

,id,created_at,price,area,street,ward,district,post_date,num_bedroom,num_diningroom,num_kitchen,num_toilet,num_floor,current_floor,direction,street_width
0,1,2024-05-12T07:05:18.963759+00:00,4.5,30,Lạc Long Quân,Xuân La,Tây Hồ,2024-05-11 00:00:00,0,0,0,0,0,0,0,0
1,2,2024-05-12T07:05:24.674404+00:00,24.0,400,Tô Hiệu,Quang Trung,Hà Đông,2024-05-11 00:00:00,0,0,0,0,5,0,0,45
2,3,2024-05-12T07:05:29.948452+00:00,4.5,30,Hàng Cháo,Cát Linh,Đống Đa,2024-05-11 00:00:00,0,0,0,0,0,0,0,0
3,4,2024-05-12T07:05:36.688087+00:00,3.6,20,Nguyễn Khuyến,Văn Quán,Hà Đông,2024-05-11 00:00:00,1,0,0,0,0,0,0,0
4,5,2024-05-12T07:05:42.626516+00:00,4.5,30,Bưởi,Vĩnh Phúc,Ba Đình,2024-05-11 00:00:00,0,0,0,0,0,0,0,0


In [5]:
missing_values = df.isnull().sum()
missing_values

id                  0
created_at          0
price               0
area                0
street            204
ward                0
district           50
post_date           0
num_bedroom         0
num_diningroom      0
num_kitchen         0
num_toilet          0
num_floor           0
current_floor       0
direction           0
street_width        0
dtype: int64

In [21]:
from scipy import stats
data = pd.read_csv('data.csv')
data['price_zscore'] = stats.zscore(data['price'])
data['area_zscore'] = stats.zscore(data['area'])
price_outliers_zscore = data[(data['price_zscore'].abs() > 3)]
area_outliers_zscore = data[(data['area_zscore'].abs() > 3)]
outliers_zscore = pd.concat([price_outliers_zscore, area_outliers_zscore]).drop_duplicates()
outliers_zscore.head()
# Remove outliers
data = data.drop(outliers_zscore.index)


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet

# Drop unnecessary columns
# data = data.drop(columns=['id', 'created_at', 'post_date'])

# Identify categorical and numerical columns
categorical_cols = ['street', 'ward', 'district', 'direction']
numerical_cols = data.drop(columns=['price'] + categorical_cols).columns.tolist()

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mean')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


# models = {
    # "LinearRegression": LinearRegression(),
    # "Lasso": Lasso(),
    # "Ridge": Ridge(),
    # "ElasticNet": ElasticNet(),
    # "RandomForestRegressor": RandomForestRegressor()
# }
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', LinearRegression())])

# Split the data into training and testing sets
X = data.drop(columns='price')
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of test data, get predictions
y_pred = clf.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')


Mean Absolute Error: 0.0005316732193177112


In [25]:
print(numerical_cols)
# data frame x_test
X_test.head()


# sample data to predict


['area', 'num_bedroom', 'num_diningroom', 'num_kitchen', 'num_toilet', 'num_floor', 'current_floor', 'street_width', 'price_zscore', 'area_zscore']


,area,street,ward,district,num_bedroom,num_diningroom,num_kitchen,num_toilet,num_floor,current_floor,direction,street_width,price_zscore,area_zscore
2411,20,Định Công Thượng,Định Công,Hoàng Mai,0,0,1,1,0,0,0,0,-0.043847,-0.413212
2455,12,Tỉnh lộ 72,An Khánh,Hoài Đức,0,0,1,0,0,0,0,0,-0.202474,-0.889560
1966,28,NaN,NaN,Láng Hạ,0,0,1,0,0,0,0,0,0.077455,0.063135
1612,35,NaN,NaN,Hoàng Mai,0,0,0,0,0,0,0,0,0.049462,0.479940
2314,20,Quan Hoa,Quan Hoa,Cầu Giấy,0,0,0,0,0,0,0,0,0.012139,-0.413212


In [26]:
y_test.head()


2411    3.0
2455    1.3
1966    4.3
1612    4.0
2314    3.6
Name: price, dtype: float64

In [27]:
y_pred[:5]

array([3.00005518, 1.30112896, 4.3001102 , 3.99925881, 3.59979732])